# KINGS COUNTY HOLDOUT DATA

In [1]:
import numpy as np
import pandas as pd


In [3]:
df = pd.read_csv('kc_house_data_test_features.csv', index_col=0)
print(df.shape)
df.head()

(4322, 20)


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [4]:
import pickle

infile = open('model.pickle','rb')
model = pickle.load(infile)
infile.close()

In [5]:
print(model.intercept_)

-232296.05402110657


In [6]:
print(len(model.coef_))

79


In [8]:
df['bedrooms'] = np.where(df['bedrooms'] == 33, 3, df['bedrooms'] )
df['bedrooms'].describe()

count    4322.000000
mean        3.387552
std         0.905875
min         0.000000
25%         3.000000
50%         3.000000
75%         4.000000
max        10.000000
Name: bedrooms, dtype: float64

In [9]:
df['age'] = 2015 - df['yr_built']

In [10]:
df['renovated']=df['yr_renovated'].apply(lambda x:1 if x>0 else 0)

In [11]:
df['has_basement'] = df['sqft_basement'].apply(lambda x:1 if x>0 else 0)

In [12]:
df.date = pd.to_datetime(df.date)
df['month'] = df.date.dt.month


In [13]:
categorical = df[['month','view','condition','yr_built','yr_renovated','zipcode','renovated','has_basement']].copy()
numerical = df[['bedrooms','sqft_living','floors','sqft_lot','sqft_basement','lat','long','sqft_living15','sqft_lot15']].copy()

In [14]:
bins = [0, 1, 4] 
bins_view = pd.cut(categorical['view'], bins, include_lowest=True, labels=['not_viewed', 'viewed'])
bins_view = bins_view.cat.as_unordered()

In [15]:
bins = [1900, 1925, 1950, 1975, 2000, 2015]
bins_yr_built = pd.cut(categorical['yr_built'], bins, include_lowest=True, labels = ('1900-1925', '1926-1950', '1951-1975', '1976-2000', '2001-2015'))
bins_yr_built = bins_yr_built.cat.as_unordered()

In [16]:
df.month =df.month.replace(12,0)
bins = [0,3,6,9,11]
bins_date = pd.cut(df['month'], bins, include_lowest=True, labels=['Winter','Summer','Spring','Fall'])
bins_date = bins_date.cat.as_unordered()

In [17]:
categorical = categorical.drop(['view', 'month', 'yr_built','yr_renovated'], axis=1)
categorical = pd.concat([categorical, bins_date, bins_view, bins_yr_built,], axis=1)

In [18]:
categorical['month'] = categorical['month'].astype('str')
categorical['view'] = categorical['view'].astype('str')
categorical['condition'] = categorical['condition'].astype('str')
categorical['zipcode'] = categorical['zipcode'].astype('str')
categorical['renovated'] = categorical['renovated'].astype('str')
categorical['has_basement'] = categorical['has_basement'].astype('str')
categorical['yr_built'] = categorical['yr_built'].astype('str')

In [19]:
categorical = pd.get_dummies(categorical)
categorical.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4322 entries, 0 to 4322
Data columns (total 90 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   condition_1         4322 non-null   uint8
 1   condition_2         4322 non-null   uint8
 2   condition_3         4322 non-null   uint8
 3   condition_4         4322 non-null   uint8
 4   condition_5         4322 non-null   uint8
 5   zipcode_98001       4322 non-null   uint8
 6   zipcode_98002       4322 non-null   uint8
 7   zipcode_98003       4322 non-null   uint8
 8   zipcode_98004       4322 non-null   uint8
 9   zipcode_98005       4322 non-null   uint8
 10  zipcode_98006       4322 non-null   uint8
 11  zipcode_98007       4322 non-null   uint8
 12  zipcode_98008       4322 non-null   uint8
 13  zipcode_98010       4322 non-null   uint8
 14  zipcode_98011       4322 non-null   uint8
 15  zipcode_98014       4322 non-null   uint8
 16  zipcode_98019       4322 non-null   uint8


In [20]:
final = pd.concat([numerical, categorical],axis=1)
final.head()

,bedrooms,sqft_living,floors,sqft_lot,sqft_basement,lat,long,sqft_living15,sqft_lot15,condition_1,...,month_Spring,month_Summer,month_Winter,view_not_viewed,view_viewed,yr_built_1900-1925,yr_built_1926-1950,yr_built_1951-1975,yr_built_1976-2000,yr_built_2001-2015
0,4,2270,1.0,11500,730,47.7089,-122.241,2020,10918,0,...,1,0,0,1,0,0,0,1,0,0
1,4,2270,1.0,11500,730,47.7089,-122.241,2020,10918,0,...,0,0,1,1,0,0,0,1,0,0
2,3,1470,2.0,1779,310,47.5472,-121.998,1470,1576,0,...,0,0,0,1,0,0,0,0,0,1
3,3,1280,1.0,16200,250,47.7427,-122.071,1160,10565,0,...,0,0,1,1,0,0,0,0,1,0
4,4,2830,2.0,8126,0,47.4863,-122.140,2830,7916,0,...,0,0,1,1,0,0,0,0,0,1


In [42]:
selected_rfe = ['has_basement_1', 'has_basement_0', 'renovated_1', 'renovated_0',
       'month_Summer', 'month_Winter', 'month_Spring', 'month_Fall',
       'sqft_lot', 'floors', 'sqft_basement', 'sqft_living15', 'sqft_lot15',
       'sqft_living', 'zipcode_98001', 'zipcode_98002', 'zipcode_98003',
       'zipcode_98004', 'zipcode_98005', 'zipcode_98006', 'zipcode_98007',
       'zipcode_98008', 'zipcode_98010', 'zipcode_98011', 'zipcode_98014',
       'zipcode_98019', 'zipcode_98022', 'zipcode_98023', 'zipcode_98024',
       'zipcode_98027', 'zipcode_98028', 'zipcode_98029', 'zipcode_98030',
       'zipcode_98031', 'zipcode_98032', 'zipcode_98033', 'zipcode_98034',
       'zipcode_98038', 'zipcode_98039', 'zipcode_98040', 'zipcode_98042',
       'zipcode_98045', 'zipcode_98052', 'zipcode_98053', 'zipcode_98055',
       'zipcode_98056', 'zipcode_98058', 'zipcode_98059', 'zipcode_98065',
       'zipcode_98070', 'zipcode_98072', 'zipcode_98074', 'zipcode_98075',
       'zipcode_98077', 'zipcode_98092', 'zipcode_98102', 'zipcode_98103',
       'zipcode_98105', 'zipcode_98106', 'zipcode_98107', 'zipcode_98108',
       'zipcode_98109', 'zipcode_98112', 'zipcode_98115', 'zipcode_98116',
       'zipcode_98117', 'zipcode_98118', 'zipcode_98119', 'zipcode_98122',
       'zipcode_98125', 'zipcode_98126', 'zipcode_98133', 'zipcode_98136',
       'zipcode_98144', 'zipcode_98146', 'zipcode_98148', 'zipcode_98155',
       'zipcode_98166', 'zipcode_98168', 'zipcode_98177', 'zipcode_98178',
       'zipcode_98188', 'zipcode_98198', 'zipcode_98199']

In [43]:
final_answers = model.predict(final[selected_rfe])

In [46]:
(final_answers)

4322

In [48]:
answers = pd.DataFrame(final_answers)

In [50]:
answers.describe()

,0
count,4.322000e+03
mean,5.674516e+05
std,3.204393e+05
min,-6.153029e+04
25%,3.503147e+05
50%,5.058219e+05
75%,7.133657e+05
max,2.716259e+06


In [49]:
answers.to_csv('housing_preds_Theron_Glynn.csv')